# Model comparison

This notebook is intended to run a model comparison on arbitrary data of the firing rate distribution obtained from the selfconsistent model (dark matter paper) vs lognormal (or other) firing rate distributions

## Steps required

- [ ] Preparation of data
- [ ] Bayesian Inference (selfconcistent)
- [ ] Bayesian Inference (log-normal)
- [ ] compare models (KL-divergence?)
- [ ] plotting

In [1]:
import os, sys
import numpy as np
root_dir = os.path.dirname(os.path.abspath(''))
if not root_dir in sys.path: sys.path.append(root_dir)
from empirical.readData import *

### Preparation of data
Data has to be brought into a consistent form, first: **Pandas dataframe**
* row=neuron-ID
* column=multicolumn
    * level 1: mouse-type
    * level 2: mouse-ID

Finally data is read into according variables

In [12]:
import pandas as pd
#import seaborn as sns

#mP_withSilent = ModelParams('empirical',filePath='../../data/BuscheLab/2P_data.xlsx',include_silent=True)

def initialize_rate_df(mouse_types,nMice,n_layer=1,n_cluster=1):
    
    ## initializing an empty pandas-dataframe to store neuron firing rate data
    # mouse_types: list(str)
    #    list of mouse types
    # nMice:       list(int)
    #    number of mice per mouse type
    # n_layer:     int [default: 1]
    #    number of layers in the data
    # n_cluster:   int [default: 1]
    #    number of neuron cluster types (exc., inh., ...)
    
    ## creating the column labels
    col_names = []
    for mouse_type in mouse_types:
        for mouse_ID in range(max(nMice)):
            for layer in range(n_layer):
                for cluster in range(n_cluster):
                    col_names.append((mouse_type,mouse_ID,layer,cluster))

    ## creating an empty dataframe
    df = pd.DataFrame(columns=col_names)
    df.columns = pd.MultiIndex.from_tuples(df.columns, names=['mouse type','mouse ID','layer','cluster'])
    
    return df

df = initialize_rate_df(mouse_types=['WT','LM','cTKO'],nMice=[3,3,4],n_layer=1,n_cluster=1)
#n_neuron = 1000
#ind_names = []
#for layer in range(n_layer):
#    for cluster in range(n_cluster):
#        for neuron_ID in range(n_neuron):
#            ind_names.append((layer,cluster,neuron_ID))

#df = pd.DataFrame(np.random.randn(n_neuron,len(mouse_types)*max(nMice)*n_layer*n_cluster),columns=col_names)
#df.index = pd.MultiIndex.from_tuples(df.index, names=(u'ID'))

In [13]:
df

Empty DataFrame
Columns: [(WT, 0, 0, 0), (WT, 0, 0, 1), (WT, 1, 0, 0), (WT, 1, 0, 1), (WT, 2, 0, 0), (WT, 2, 0, 1), (WT, 3, 0, 0), (WT, 3, 0, 1), (LM, 0, 0, 0), (LM, 0, 0, 1), (LM, 1, 0, 0), (LM, 1, 0, 1), (LM, 2, 0, 0), (LM, 2, 0, 1), (LM, 3, 0, 0), (LM, 3, 0, 1), (cTKO, 0, 0, 0), (cTKO, 0, 0, 1), (cTKO, 1, 0, 0), (cTKO, 1, 0, 1), (cTKO, 2, 0, 0), (cTKO, 2, 0, 1), (cTKO, 3, 0, 0), (cTKO, 3, 0, 1)]
Index: []

[0 rows x 24 columns]

In [5]:
mP_withSilent.rates_raw.shape

(13, 1, 1, 370)

In [36]:
mP.rates_raw

df.xs('WT',level='mouse type',axis=1)
df

mouse type        WT                  LM                cTKO          
mouse ID           0         1         0         1         0         1
layer              0         0         0         0         0         0
cluster            0         0         0         0         0         0
0          -0.821566  0.372431 -0.992036  0.107506  0.786820  0.272716
1          -0.035196 -1.127935  0.787413 -0.166839  0.306910 -0.706743
2           1.877915 -0.339093 -1.641550  0.062607  1.038418  0.356631
3          -1.057294  0.200217  0.005318 -0.829674  0.290564 -0.494596
4          -0.646761 -2.154097 -0.704697  0.975074  0.601389  2.032919
..               ...       ...       ...       ...       ...       ...
995        -0.772357 -0.452172 -1.528291 -1.367829 -1.134900 -0.922199
996         0.698880  0.455069 -0.584202  1.347918  0.932154 -0.793963
997         0.352662  1.549799  0.431776 -0.489599  0.157914  1.391942
998        -0.819327  0.442166 -0.757770 -0.507246 -0.321333 -2.108448
999         0.994412  0.351766 -1.407336  1.021817  0.236158  2.390859

[1000 rows x 6 columns]

In [9]:
mP_withSilent.spikes_raw.shape

(13, 1, 1, 370)

### Bayesian Inference (selfconsistent)
Running inference (pymc3, H(amiltonian)MC) on data, inferring best parameters

### Bayesian Inference (log-normal)
Running inference (pymc3, H(amiltonian)MC) on data, inferring best parameters